# Natural Statistics Cross-linguistic: 

#### Lexical diversity analysis - random sample 

----

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "data_proc")
import analytic_proc
import warnings
warnings.filterwarnings('ignore')

In [2]:
rand_dat_inc = pd.read_csv("../data/rand_dat_inc_master.csv",index_col=0,low_memory=False)
rand_dat_inc=rand_dat_inc[rand_dat_inc["language"]!="ara"]
rand_dat_inc=rand_dat_inc[(rand_dat_inc["target_child_age"]>=5) & (rand_dat_inc["target_child_age"]<=30)]

rand_dat_inc_cg = rand_dat_inc[rand_dat_inc["caregiver"]=="caregiver"]

rand_dat_inc_cg["contingent"] = np.where(rand_dat_inc_cg["contingent"]==1, "contingent", "non-contingent")

rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"].notna()]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="xxx"]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="yyy"]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="www"]

Would call the following if we could run lexical diversity analysis globally:

```python
analytic_proc.create_result(rand_dat_inc_cg)
```

However, we want to have seperate dictionaries for contingent and non-contingent words so we can compare them to one another.

The function will allow us to have a different dictionary for each transcript.

Finally, to compare, we can run mixed effects to understand whether contingent and non-contingent utterances differ in their lexical diversity, controlling for number of transcripts.

----
#### Seperate contingent and non-contingent utterances into individual dataframes

In [3]:
rand_dat_inc_cg_cc = rand_dat_inc_cg[rand_dat_inc_cg["contingent"]=="contingent"].reset_index(drop=True)
rand_dat_inc_cg_nc = rand_dat_inc_cg[rand_dat_inc_cg["contingent"]=="non-contingent"].reset_index(drop=True)

----
#### Loop through each unique transcript to compute the lexical diversity counts across languages.

In [4]:
# analytic_proc.create_result(rand_dat_inc_cg_cc)

In [5]:
# change the "to_csv" line at the end of analytic_proc.py first

# analytic_proc.create_result(rand_dat_inc_cg_nc)

----
#### Lexical Diversity plot

In [6]:
rand_dat_inc_cg_cc = pd.read_csv("../data/rand_dat_inc_master_cc_lexdiv.csv",index_col=0,low_memory=False)
rand_dat_inc_cg_nc = pd.read_csv("../data/rand_dat_inc_master_nc_lexdiv.csv",index_col=0,low_memory=False)

# combine dataframes into one

rand_dat_inc_cg = pd.concat([rand_dat_inc_cg_cc,rand_dat_inc_cg_nc])

In [7]:
ar = [['min', rand_dat_inc_cg["target_child_age"].min()],
      ['max', rand_dat_inc_cg["target_child_age"].max()],
      ['mean', rand_dat_inc_cg["target_child_age"].mean()],
      ['stdev', rand_dat_inc_cg["target_child_age"].std()]]

age_range = pd.DataFrame(ar, columns = ['stat', 'age'])

print(age_range.to_markdown())

|    | stat   |      age |
|---:|:-------|---------:|
|  0 | min    |  5.98575 |
|  1 | max    | 29.9862  |
|  2 | mean   | 18.6737  |
|  3 | stdev  |  6.83431 |


In [8]:
rand_lex_stats = (rand_dat_inc_cg.groupby(["Language_name","target_child_id","transcript_id","contingent"])
                                  .uniquenss
                                  .agg(["sum"])
                                  .reset_index())
rand_lex_sumstats =  rand_lex_stats.rename({'sum': 'sums'}, axis=1)

In [9]:
len(rand_lex_sumstats["Language_name"].unique())

14

In [10]:
len(rand_lex_sumstats["transcript_id"].unique())

1690

In [11]:
rand_lex_sumstats.to_csv("../data/rand_lex_sumstats.csv")

In [12]:
%load_ext rpy2.ipython

Simple plot

In [13]:
%%R -i rand_lex_sumstats

library('ggplot2')
library('repr')
options(repr.plot.width=6, repr.plot.height=12)

xlabs <- c("C", "NC")

# ara_label <- data.frame(means=c(0),contingent = c(1.5),language="ara") # no adult speech transcribed
deu_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="English")
est_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Estonian")
fas_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Korean")
# nor_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Norwegian")
nor_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Norwegian")
pol_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Portuguese")
spa_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Spanish")
swe_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Swedish")
zho_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Mandarin")
# zho_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(rand_lex_sumstats, aes(x = contingent, y = sums, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=8,color="black") + 
     geom_text(data = eng_label,label = "***",size=8,color="black") +  
     geom_text(data = est_label,label = "**",size=8,color="black") +  
#      geom_text(data = fas_ns_label,label = "*",size=4,color="black",fontface = "italic") +
     geom_text(data = fas_label,label = "*",size=8, color="black") +
     geom_text(data = fra_label,label = "***",size=8,color="black") +  
     geom_text(data = hrv_label,label = "***",size=8,color="black") + 
     geom_text(data = jpn_label,label = "***",size=8,color="black") + 
     geom_text(data = kor_label,label = "***",size=8,color="black") +  
     geom_text(data = nor_ns_label,label = "ns",size=4,color="black",fontface = "italic") +  
#      geom_text(data = pol_ns_label,label = "ns", size=4,color="black",fontface = "italic") +  
     geom_text(data = por_label,label = "**",size=8,color="black") +  
     geom_text(data = spa_ns_label,label = "ns",size=4,color="black",fontface = "italic") + 
     geom_text(data = swe_label,label = "ns",size=4,color="black",fontface = "italic") + 
#      geom_text(data = zho_ns_label,label = "ns",size=4,color="black",fontface = "italic") +
     geom_text(data = zho_label,label = "^",size=8, color="black") +
     ylim(0, 250) +
     labs(tag = "A",
          y = "Number of Unique Words", x = "") +
     theme_classic() +
     scale_x_discrete(labels= xlabs) +
     theme(text = element_text(size=16),
           axis.text.x = element_text(vjust = 0.5, hjust=.5),
           legend.title = element_blank(),
           legend.background = element_rect(fill=alpha("white",0.90),
                                                            size=0, linetype="dotted",
                                                            colour = "white"),
           legend.text=element_text(size=16))
     ggsave("../figures/lexical_diversity_rand.pdf", width = 11.7, height = 6.2)

for manuscript

In [14]:
%%R -i rand_lex_sumstats

library('ggplot2')
library('repr')
options(repr.plot.width=6, repr.plot.height=12)

xlabs <- c("C", "NC")

# ara_label <- data.frame(means=c(0),contingent = c(1.5),language="ara") # no adult speech transcribed
deu_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="English")
est_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Estonian")
fas_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Korean")
# nor_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Norwegian")
nor_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Norwegian")
pol_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Portuguese")
spa_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Spanish")
swe_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Swedish")
zho_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Mandarin")
# zho_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(rand_lex_sumstats, aes(x = contingent, y = sums, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .4) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=6,color="black") + 
     geom_text(data = eng_label,label = "***",size=6,color="black") +  
     geom_text(data = est_label,label = "**",size=6,color="black") +  
     geom_text(data = fas_label,label = "*",size=6, color="black") +
     geom_text(data = fra_label,label = "***",size=6,color="black") +  
     geom_text(data = hrv_label,label = "***",size=6,color="black") + 
     geom_text(data = jpn_label,label = "***",size=6,color="black") + 
     geom_text(data = kor_label,label = "***",size=6,color="black") +  
     geom_text(data = nor_ns_label,label = "ns",size=3,color="black",fontface = "italic") +  
     geom_text(data = por_label,label = "**",size=6,color="black") +  
     geom_text(data = spa_ns_label,label = "ns",size=3,color="black",fontface = "italic") + 
     geom_text(data = swe_label,label = "ns",size=3,color="black",fontface = "italic") + 
     geom_text(data = zho_label,label = "^",size=3, color="black") +
     ylim(0, 250) +
     labs(tag = "A",
          y = "Number of Unique Words", x = "") +
     theme_classic() +
     scale_x_discrete(labels= xlabs) +
     theme(text = element_text(size=11.5),
           axis.text.x = element_text(vjust = 0.5, hjust=0.5),
           legend.position="none")
     ggsave("../figures/figure_2_A.pdf", width = 11.5, height = 4.2)

Plot + effect estimates

In [15]:
%%R

deu_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="German")
eng_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="English")
est_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Estonian")
fas_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Persian")
fra_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="French")
hrv_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Croatian")
jpn_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Japanese")
kor_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Korean")
nor_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Norwegian")
por_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Portuguese")
spa_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Spanish")
swe_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Swedish")
zho_est_label <- data.frame(sums=c(.25),contingent = c(1),Language_name="Mandarin")

p <- p + geom_text(data = deu_est_label,label = "est=-66",size=4,color="black") +
         geom_text(data = eng_est_label,label = "est=-122",size=4,color="black") +
         geom_text(data = est_est_label,label = "est=-54.2",size=4,color="black") +
         geom_text(data = fas_est_label,label = "est=-51.1",size=4,color="black") +
         geom_text(data = fra_est_label,label = "est=-67.3",size=4,color="black") +
         geom_text(data = hrv_est_label,label = "est=-37.5",size=4,color="black") +
         geom_text(data = jpn_est_label,label = "est=-41.9",size=4,color="black") +
         geom_text(data = kor_est_label,label = "est=-101",size=4,color="black") +
#          geom_text(data = nor_est_label,label = "est=-.68",size=4,color="black") +
         geom_text(data = por_est_label,label = "est=-32.5",size=4,color="black") +
#          geom_text(data = spa_est_label,label = "est=-.39",size=4,color="black") +
         geom_text(data = swe_est_label,label = "est=-43.4",size=4,color="black") +
         geom_text(data = zho_est_label,label = "est=-60",size=4,color="black")
         

ggsave("../figures/lexical_diversity_rand_eff.pdf", width = 11.7, height = 6.2)

\+ sample size

In [16]:
%%R

deu_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="German")
eng_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="English")
est_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Estonian")
fas_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Persian")
fra_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="French")
hrv_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Croatian")
jpn_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Japanese")
kor_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Korean")
nor_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Norwegian")
pol_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Polish")
por_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Portuguese")
spa_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Spanish")
swe_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Swedish")
zho_n_label <- data.frame(sums=c(.25),contingent = c(1.7),Language_name="Mandarin")

deu_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="German")
eng_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="English")
est_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Estonian")
fas_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Persian")
fra_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="French")
hrv_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Croatian")
jpn_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Japanese")
kor_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Korean")
nor_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Norwegian")
pol_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Polish")
por_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Portuguese")
spa_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Spanish")
swe_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Swedish")
zho_sz_label <- data.frame(sums=c(.25),contingent = c(2.1),Language_name="Mandarin")

p <- p + geom_text(data = deu_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = eng_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = est_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = fas_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = fra_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = hrv_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = jpn_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = kor_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = nor_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = pol_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = por_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = spa_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = swe_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = zho_n_label,label = "n",size=4,color="black",fontface = "italic") +
         geom_text(data = deu_sz_label,label = " = 39",size=4,color="black") +
         geom_text(data = eng_sz_label,label = " = 1005",size=4,color="black") +
         geom_text(data = est_sz_label,label = " = 22",size=4,color="black") +
         geom_text(data = fas_sz_label,label = " = 12",size=4,color="black") +
         geom_text(data = fra_sz_label,label = " = 303",size=4,color="black") +
         geom_text(data = hrv_sz_label,label = " = 79",size=4,color="black") +
         geom_text(data = jpn_sz_label,label = " = 139",size=4,color="black") +
         geom_text(data = kor_sz_label,label = " = 37",size=4,color="black") +
         geom_text(data = nor_sz_label,label = " = 56",size=4,color="black") +
         geom_text(data = pol_sz_label,label = " = 1",size=4,color="black") +
         geom_text(data = por_sz_label,label = " = 24",size=4,color="black") +
         geom_text(data = spa_sz_label,label = " = 31",size=4,color="black") +
         geom_text(data = swe_sz_label,label = " = 16",size=4,color="black") +
         geom_text(data = zho_sz_label,label = " = 2",size=4,color="black")
         

ggsave("../figures/lexical_diversity_rand_eff_n.pdf", width = 11.7, height = 6.2)

----
#### Lexical diversity mixed models

In [17]:
# ara=rand_dat_inc_cg[['language','num_tokens','contingent','transcript_id','target_child_id']][rand_dat_inc_cg["language"]=="ara"] # no adult speech transcribed
deu=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="German"]
eng=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="English"]
est=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Estonian"]
fas=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Persian"]
fra=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="French"]
hrv=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Croatian"]
jpn=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Japanese"]
kor=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Korean"]
nor=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Norwegian"]
pol=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Polish"]
por=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Portuguese"]
spa=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Spanish"]
swe=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Swedish"]
zho=rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id']][rand_lex_sumstats["Language_name"]=="Mandarin"]

In [18]:
%%R

library("lme4")
library("emmeans")
library("lmerTest")
library("tidyverse")

effect_sizes <- data.frame(matrix(ncol = 2, nrow = 0))
cols <- c("Language_name", "rand_effect_size")
colnames(effect_sizes) <- cols

R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step


R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1
✔ purrr   0.3.4     

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()



In [19]:
%%R -i deu

lm2 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=deu, REML= FALSE)
emm2<-emmeans(lm2,pairwise~contingent)
pval<-summary(emm2$contrasts)$p.value
print(c(emm2$contrasts, pval))
p.adjust(pval, "holm", 14) # create big vector of p-values and ajdust those
# summary(emmeans(lm2,"contingent",contr="pairwise"),infer=TRUE) #group means
# test(contrast(emmeans(lm2,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

deu_lname <- deu$Language_name[1]

deu_eff <- eff_size(emm2,sigma = sigma(lm2), edf = df.residual(lm2))

deu_eff <- summary(deu_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(deu_lname,deu_eff)
effect_sizes

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)      -62 7.81 38  -7.946  <.0001

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 1.340226e-09



R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1        German -1.82300800568462


In [20]:
%%R -i eng

lm3 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=eng, REML= FALSE)
emm3<-emmeans(lm3,pairwise~contingent)
pval<-summary(emm3$contrasts)$p.value
print(c(emm3$contrasts, pval))
p.adjust(pval, "holm", 14)
# summary(emmeans(lm3,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm3,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

eng_lname <- eng$Language_name[1]

eng_eff <- eff_size(emm3,sigma = sigma(lm3), edf = df.residual(lm3))

eng_eff <- summary(eng_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(eng_lname,eng_eff)
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE   df t.ratio p.value
 contingent - (non-contingent)     -108 2.02 1676 -53.506  <.0001

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 1.49647e-12



R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1        German -1.82300800568462
2       English -2.46968244455422


In [21]:
%%R -i est

lm4 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=est, REML= FALSE)
emm4<-emmeans(lm4,pairwise~contingent)
pval<-summary(emm4$contrasts)$p.value
print(c(emm4$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm4,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm4,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

est_lname <- est$Language_name[1]

est_eff <- eff_size(emm4,sigma = sigma(lm4), edf = df.residual(lm4))

est_eff <- summary(est_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(est_lname,est_eff)
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)      -59 13.6 22  -4.329  0.0003

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 0.0002698171

[1] 0.003777439


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1        German -1.82300800568462
2       English -2.46968244455422
3      Estonian  -1.3053776625777


In [22]:
%%R -i fas

lm5 <- lmer(sums ~ contingent + (1|transcript_id),data=fas, REML= FALSE)
emm5<-emmeans(lm5,pairwise~contingent)
pval<-summary(emm5$contrasts)$p.value
print(c(emm5$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm5,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm5,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

fas_lname <- fas$Language_name[1]

fas_eff <- eff_size(emm5,sigma = sigma(lm5), edf = df.residual(lm5))

fas_eff <- summary(fas_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(fas_lname,fas_eff)
effect_sizes

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)      -67 10.2 10  -6.556  0.0001

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 6.423789e-05

[1] 0.0008993305


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1        German -1.82300800568462
2       English -2.46968244455422
3      Estonian  -1.3053776625777
4       Persian -2.93194866482665


In [23]:
%%R -i fra

lm6 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=fra, REML= FALSE)
emm6<-emmeans(lm6,pairwise~contingent)
pval<-summary(emm6$contrasts)$p.value
print(c(emm6$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm6,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm6,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

fra_lname <- fra$Language_name[1]

fra_eff <- eff_size(emm6,sigma = sigma(lm6), edf = df.residual(lm6))

fra_eff <- summary(fra_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(fra_lname,fra_eff)
effect_sizes

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE  df t.ratio p.value
 contingent - (non-contingent)    -68.9 3.35 279 -20.583  <.0001

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 3.597123e-13

[1] 5.035972e-12


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1        German -1.82300800568462
2       English -2.46968244455422
3      Estonian  -1.3053776625777
4       Persian -2.93194866482665
5        French -1.73890442295843


In [24]:
%%R -i hrv

lm7 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=hrv, REML= FALSE)
emm7<-emmeans(lm7,pairwise~contingent)
pval<-summary(emm7$contrasts)$p.value
print(c(emm7$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm7,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm7,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

hrv_lname <- hrv$Language_name[1]

hrv_eff <- eff_size(emm7,sigma = sigma(lm7), edf = df.residual(lm7))

hrv_eff <- summary(hrv_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(hrv_lname,hrv_eff)
effect_sizes

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)    -40.2 4.26 56  -9.437  <.0001

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 7.681744e-12

[1] 1.075444e-10


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1        German -1.82300800568462
2       English -2.46968244455422
3      Estonian  -1.3053776625777
4       Persian -2.93194866482665
5        French -1.73890442295843
6      Croatian -1.78338336898637


In [25]:
%%R -i jpn

lm8 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=jpn, REML= FALSE)
emm8<-emmeans(lm8,pairwise~contingent)
pval<-summary(emm8$contrasts)$p.value
print(c(emm8$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm8,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm8,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

jpn_lname <- jpn$Language_name[1]

jpn_eff <- eff_size(emm8,sigma = sigma(lm8), edf = df.residual(lm8))

jpn_eff <- summary(jpn_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(jpn_lname,jpn_eff)
effect_sizes

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE  df t.ratio p.value
 contingent - (non-contingent)    -42.7 3.03 139 -14.064  <.0001

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 0

[1] 0


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1        German -1.82300800568462
2       English -2.46968244455422
3      Estonian  -1.3053776625777
4       Persian -2.93194866482665
5        French -1.73890442295843
6      Croatian -1.78338336898637
7      Japanese -1.68699142881437


In [26]:
%%R -i kor

lm9 <- lmer(sums ~ contingent + (1|transcript_id), data=kor, REML= FALSE)
emm9<-emmeans(lm9,pairwise~contingent)
pval<-summary(emm9$contrasts)$p.value
print(c(emm9$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm9,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm9,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

kor_lname <- kor$Language_name[1]

kor_eff <- eff_size(emm9,sigma = sigma(lm9), edf = df.residual(lm9))

kor_eff <- summary(kor_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(kor_lname,kor_eff)
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)     -112 13.9 56  -8.054  <.0001

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 7.023804e-11

[1] 9.833325e-10


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



  Language_name  rand_effect_size
1        German -1.82300800568462
2       English -2.46968244455422
3      Estonian  -1.3053776625777
4       Persian -2.93194866482665
5        French -1.73890442295843
6      Croatian -1.78338336898637
7      Japanese -1.68699142881437
8        Korean -2.15263727391117


In [27]:
%%R -i nor

lm10 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=nor, REML= FALSE)
emm10<-emmeans(lm10,pairwise~contingent)
pval<-summary(emm10$contrasts)$p.value
print(c(emm10$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm10,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm10,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

nor_lname <- nor$Language_name[1]

effect_sizes[nrow(effect_sizes)+1,] <- c(nor_lname,NaN)
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE   df t.ratio p.value
 contingent - (non-contingent)    -22.9 8.35 37.1  -2.739  0.0094

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 0.009417298

[1] 0.1318422
  Language_name  rand_effect_size
1        German -1.82300800568462
2       English -2.46968244455422
3      Estonian  -1.3053776625777
4       Persian -2.93194866482665
5        French -1.73890442295843
6      Croatian -1.78338336898637
7      Japanese -1.68699142881437
8        Korean -2.15263727391117
9     Norwegian               NaN


In [28]:
%%R -i pol

# simple linear model (no random effects, because only 1 transcript from 1 sub)

#

lm11 <- lm(sums ~ contingent ,data=pol, REML= FALSE)
emm11<-emmeans(lm11,pairwise~contingent)
pval<-summary(emm11$contrasts)$p.value
print(c(emm11$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm11,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm11,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

pol_lname <- pol$Language_name[1]

effect_sizes[nrow(effect_sizes)+1,] <- c(pol_lname,NaN)
effect_sizes

[[1]]
 contrast                      estimate  SE df t.ratio p.value
 contingent - (non-contingent)      -21 NaN  0     NaN     NaN


[[2]]
[1] NaN

[1] NaN
   Language_name  rand_effect_size
1         German -1.82300800568462
2        English -2.46968244455422
3       Estonian  -1.3053776625777
4        Persian -2.93194866482665
5         French -1.73890442295843
6       Croatian -1.78338336898637
7       Japanese -1.68699142881437
8         Korean -2.15263727391117
9      Norwegian               NaN
10        Polish               NaN


In [29]:
%%R -i por

lm12 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=por, REML= FALSE)
emm12<-emmeans(lm12,pairwise~contingent)
pval<-summary(emm12$contrasts)$p.value
print(c(emm12$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm12,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm12,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

por_lname <- por$Language_name[1]

por_eff <- eff_size(emm12,sigma = sigma(lm12), edf = df.residual(lm12))

por_eff <- summary(por_eff)$effect.size

effect_sizes[nrow(effect_sizes)+1,] <- c(por_lname,por_eff)
effect_sizes

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate  SE df t.ratio p.value
 contingent - (non-contingent)    -44.8 9.4 23  -4.766  0.0001

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 8.350929e-05

[1] 0.00116913


R[write to console]: Since 'object' is a list, we are using the contrasts already present.



   Language_name  rand_effect_size
1         German -1.82300800568462
2        English -2.46968244455422
3       Estonian  -1.3053776625777
4        Persian -2.93194866482665
5         French -1.73890442295843
6       Croatian -1.78338336898637
7       Japanese -1.68699142881437
8         Korean -2.15263727391117
9      Norwegian               NaN
10        Polish               NaN
11    Portuguese  -1.4052872748849


In [30]:
%%R -i spa

lm13 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=spa, REML= FALSE)
emm13<-emmeans(lm13,pairwise~contingent)
pval<-summary(emm13$contrasts)$p.value
print(c(emm13$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm13,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm13,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

spa_lname <- spa$Language_name[1]

effect_sizes[nrow(effect_sizes)+1,] <- c(spa_lname,NaN)
effect_sizes

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)    -3.97 5.12 31  -0.776  0.4438

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 0.4438368

[1] 1
   Language_name  rand_effect_size
1         German -1.82300800568462
2        English -2.46968244455422
3       Estonian  -1.3053776625777
4        Persian -2.93194866482665
5         French -1.73890442295843
6       Croatian -1.78338336898637
7       Japanese -1.68699142881437
8         Korean -2.15263727391117
9      Norwegian               NaN
10        Polish               NaN
11    Portuguese  -1.4052872748849
12       Spanish               NaN


In [31]:
%%R -i swe

lm14 <- lmer(sums ~ contingent + (1|target_child_id) + (1|transcript_id),data=swe, REML= FALSE)
emm14<-emmeans(lm14,pairwise~contingent)
pval<-summary(emm14$contrasts)$p.value
print(c(emm14$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm14,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm14,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

swe_lname <- swe$Language_name[1]

effect_sizes[nrow(effect_sizes)+1,] <- c(swe_lname,NaN)
effect_sizes

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)    -37.1 12.9 16  -2.882  0.0108

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 0.01083042

[1] 0.1516259
   Language_name  rand_effect_size
1         German -1.82300800568462
2        English -2.46968244455422
3       Estonian  -1.3053776625777
4        Persian -2.93194866482665
5         French -1.73890442295843
6       Croatian -1.78338336898637
7       Japanese -1.68699142881437
8         Korean -2.15263727391117
9      Norwegian               NaN
10        Polish               NaN
11    Portuguese  -1.4052872748849
12       Spanish               NaN
13       Swedish               NaN


In [32]:
%%R -i zho

lm15 <- lmer(sums ~ contingent + (1|transcript_id),data=zho, REML= FALSE)
emm15<-emmeans(lm15,pairwise~contingent)
pval<-summary(emm15$contrasts)$p.value
print(c(emm15$contrasts, pval))
print(p.adjust(pval, "holm", 14))
# summary(emmeans(lm15,"contingent",infer=TRUE)) #group means
# test(contrast(emmeans(lm15,"contingent"), "trt.vs.ctrl"), joint = TRUE) #main effect - are any groups different

# zho_lname <- zho$Language_name[1]

# zho_eff <- eff_size(emm15,sigma = sigma(lm15), edf = df.residual(lm15))

# zho_eff <- summary(zho_eff)$effect.size

# effect_sizes[nrow(effect_sizes)+1,] <- c(zho_lname,zho_eff)
# effect_sizes

R[write to console]: Cannot use mode = "kenward-roger" because *pbkrtest* package is not installed



[[1]]
 contrast                      estimate   SE df t.ratio p.value
 contingent - (non-contingent)      -92 12.7  2  -7.228  0.0185

Degrees-of-freedom method: satterthwaite 

[[2]]
[1] 0.01847086

[1] 0.258592


In [33]:
%%R
write.csv(x=effect_sizes,'../data/lexdiv_effect_sizes.csv', row.names = FALSE)